In [1]:
from diffusers import AutoencoderKL
import torch
import os
import numpy as np
import librosa
from tqdm import tqdm
import soundfile as sf
import torchaudio
import json
from IPython.display import Audio
import matplotlib.pyplot as plt

In [ ]:
vae = AutoencoderKL.from_pretrained("/blob/v-yuancwang/AudioEditingModel/VAE_GAN/checkpoint-40000/vae")
vae.requires_grad_(False)
torch_device = torch.device("cuda:1")
vae.to(torch_device)

In [5]:
train_json = "/home/v-yuancwang/AUDIT_v2/medata_infos/soundbible.json"
with open(train_json, "r") as f:
    train_list = json.load(f)
train_list[:2]

[{'mel': '/blob/v-yuancwang/WavCaps/SoundBible/mel/1010.npy',
  'caption': 'A spotted owl is making a sound.'},
 {'mel': '/blob/v-yuancwang/WavCaps/SoundBible/mel/1005.npy',
  'caption': 'A skillet hits someone in the head, great for cartoon or movie scene.'}]

In [6]:
vae_lists = []

for info in tqdm(train_list[:]):
    mel_path = info["mel"]
    wav_path = mel_path.replace("/mel/", "/wav/").replace(".npy", ".wav")
    wav, sr = librosa.load(wav_path, sr=16000)
    if len(wav) < 159500:
        continue
    mel = np.load(mel_path)

    test_mel_tensor = np.expand_dims(np.expand_dims(mel[:624], 0), 0)
    test_mel_tensor = torch.Tensor(test_mel_tensor).to(torch_device)
    with torch.no_grad():
        posterior = vae.encode(test_mel_tensor).latent_dist
        z = posterior.sample()
        vae_output = vae.decode(z).sample
    vae_res = vae_output[0][0].cpu().numpy()

    np.save(mel_path.replace("/mel/", "/vae_mel/"), vae_res)
    vae_lists.append(info)

vae_json = train_json.replace("/medata_infos/","/hifigan_ft_infos/")
with open(vae_json, "w") as f:
    json.dump(vae_lists, f)


100%|██████████| 1232/1232 [01:54<00:00, 10.79it/s]
